In [54]:
import numpy as np 
import open3d as o3d 
import mrob 

In [55]:
from sklearn.cluster import AgglomerativeClustering

In [56]:
import sys
import pathlib
import copy

sys.path.append("..")

In [57]:
import voxel_slam

In [58]:
def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [59]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"
imu_path = "/home/ach/Desktop/datasets/hilti/hilti_imu.txt"

In [60]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)
ts_to_imu = parse_trajectories_float(imu_path, ts_multiplier=ts_multiplier)

In [61]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=320, number_of_clouds=5, center_distance_threshold=3)

In [62]:
voxel_map = voxel_slam.VoxelFeatureMap(clouds, poses, voxel_size=2.0)
feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

In [63]:
len(feature_map)

84

In [64]:
voxel_slam.EmptyVoxelsFilter(min_voxel_poses=len(poses)).filter(feature_map)

In [65]:
len(feature_map)

48

In [66]:
def get_inconsistent_voxels(feature_map):
    inconsistent_voxels = []

    for voxel_id, pose_to_points in feature_map.items():
        normals = []
        for pose_id, feature_points in pose_to_points.items():
            normals.append(feature_points.get_plane_equation()[:-1])
        
        clustering = AgglomerativeClustering(
            n_clusters=None,
            distance_threshold=0.2,
            metric="cosine",
            linkage="single",
            compute_distances=True,
        ).fit(np.asarray(normals))

        if clustering.n_clusters_ > 1:
            inconsistent_voxels.append(voxel_id)

    return inconsistent_voxels

In [67]:
inconsistent_voxels = get_inconsistent_voxels(feature_map)

In [68]:
print("Total number of voxels:", len(feature_map))
print("Number of inconsistent voxels:", len(inconsistent_voxels))

Total number of voxels: 48
Number of inconsistent voxels: 8


In [69]:
def get_bounding_box(voxel_center, voxel_size):
    bounds = []
    a = voxel_size / 2
    for x in range(2):
        for y in range(2):
            for z in range(2):
                b_box = (voxel_center[0] + a * (-1 if x == 0 else 1),
                         voxel_center[1] + a * (-1 if y == 0 else 1),
                         voxel_center[2] + a * (-1 if z == 0 else 1))
                bounds.append(b_box)

    return bounds

In [70]:
def get_box_centroid(bounding_box: np.ndarray):
    return np.apply_along_axis(lambda x: (min(x) + max(x)) / 2, 0, bounding_box)

In [71]:
def point_is_in_box(point, bounding_box):
    bounding_box = np.asarray(bounding_box)
    is_in_box = True 
    for i in range(3):
        is_in_box &= min(bounding_box[:, i]) <= point[i] <= max(bounding_box[:, i]) 

    return is_in_box

In [72]:
def break_inconsistent(voxel_map: voxel_slam.VoxelFeatureMap, inconsistent_voxels):
    voxel_to_pose_points_map = voxel_map._voxel_to_pose_points_map
    current_voxel_size = voxel_map.voxel_size
    octant_size = current_voxel_size / 2
    for voxel_center in inconsistent_voxels:
        octant_centers = get_bounding_box(voxel_center, octant_size)
        # Add octant-voxels to voxel_map 
        for oct_center in octant_centers:
            if oct_center not in voxel_to_pose_points_map:
                voxel_to_pose_points_map[tuple(oct_center)] = {}
        
        # Assign points to octants
        for pose_id, voxel_points in voxel_to_pose_points_map[voxel_center].items():
            for point, point_id in zip(voxel_points.points, voxel_points.pcd_idx):
                # Find point's octant
                for oct_center in octant_centers:
                    if point_is_in_box(point, bounding_box=get_bounding_box(oct_center, octant_size)):
                        octo_points: voxel_slam.PCDPlane = voxel_to_pose_points_map[oct_center].get(pose_id, voxel_slam.PCDPlane([], []))
                        octo_points.add_point(point, point_id)
                        voxel_to_pose_points_map[oct_center].update({pose_id: octo_points})

        # Pop old voxel center 
        voxel_to_pose_points_map.pop(voxel_center)
            

In [73]:
print("Voxel Map size:", len(voxel_map._voxel_to_pose_points_map))

Voxel Map size: 84


In [74]:
break_inconsistent(voxel_map, inconsistent_voxels)

In [75]:
print("Voxel Map size:", len(voxel_map._voxel_to_pose_points_map))
print("Inconsistent size", len(inconsistent_voxels))

Voxel Map size: 140
Inconsistent size 8


In [76]:
octo_feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

In [77]:
print("Length octo feature map", len(octo_feature_map))

Length octo feature map 140


In [78]:
voxel_slam.EmptyVoxelsFilter(min_voxel_poses=len(poses)).filter(octo_feature_map)

In [79]:
print("Length octo feature map", len(octo_feature_map))

Length octo feature map 43


In [80]:
colored_clouds, color_to_voxel = voxel_map.get_colored_feature_clouds(octo_feature_map)

In [81]:
o3d.visualization.draw_geometries([colored_clouds[0]])

In [82]:
octo_inconsistent = get_inconsistent_voxels(octo_feature_map)

In [83]:
len(octo_inconsistent)

2

## Test optimization

In [84]:
def break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_iteration=0):
    transformed_clouds = [None for _ in range(len(poses))]
    for i in range(len(poses)):
        transformed_clouds[i] = copy.deepcopy(clouds[i]).transform(poses[i])

    optimized_submaps = []
    for i in range(0, len(poses), minimap_size):
        voxel_map = voxel_slam.VoxelFeatureMap(
            transformed_clouds[i:i+minimap_size],
            [np.eye(4) for _ in range(minimap_size)],
            voxel_size=2.0
        )
        feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

        for _ in range(adaptive_voxelisation_iteration):
            voxel_slam.EmptyVoxelsFilter(min_voxel_poses=minimap_size).filter(feature_map)
            inconsistent_voxels = get_inconsistent_voxels(feature_map)
            break_inconsistent(voxel_map, inconsistent_voxels)
            feature_map = voxel_map.extract_voxel_features(ransac_distance_threshold=0.02)

        print(f"Submap {i}-{i+minimap_size}:", end=' ')
        opt_poses, is_converged, chi2 = voxel_slam.BaregBackend(feature_map, minimap_size).get_optimized_poses(1000, verbose=True)
        

        optimized_submaps.append(
            voxel_slam.aggregate_map(voxel_map.get_colored_feature_clouds(feature_map)[0], opt_poses)
        )

    aggregate_filter = voxel_slam.EmptyVoxelsFilter(min_voxel_poses=2)

    print("Aggregated map:", end=' ')
    aggregate_pipeline = voxel_slam.VoxelSLAMPipeline(
        feature_filter=aggregate_filter,
        optimization_backend=voxel_slam.BaregBackend,
        config=voxel_slam.PipelineConfig(voxel_size=2.0, 
                                         ransac_distance_threshold=0.02, 
                                         filter_cosine_distance_threshold=0.2,
                                         backend_verbose=True)
    )

    aggregate_output = aggregate_pipeline.process(optimized_submaps, [np.eye(4) for _ in range(len(optimized_submaps))])

    o3d.visualization.draw_geometries([
        voxel_slam.aggregate_map(aggregate_output.optimized_clouds, aggregate_output.optimized_poses)
    ])

In [85]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=300, number_of_clouds=60, center_distance_threshold=3)

In [86]:
break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_iteration=0)

Submap 0-5: FGraph initial error: 5291.338644269284
Iteratios to converge: 19
Chi2: 5249.787217306715
Submap 5-10: FGraph initial error: 4343.404508636594
Iteratios to converge: 50
Chi2: 4324.000011988296
Submap 10-15: FGraph initial error: 37237.4077294608
Iteratios to converge: 55
Chi2: 34071.197900043444
Submap 15-20: FGraph initial error: 28172.02363297886
Iteratios to converge: 65
Chi2: 18071.091709983295
Submap 20-25: FGraph initial error: 22753.193612261017
Iteratios to converge: 71
Chi2: 15557.856515367312
Submap 25-30: FGraph initial error: 118306.9903266087
Iteratios to converge: 61
Chi2: 111391.31462955617
Submap 30-35: FGraph initial error: 503575.11240038206
Iteratios to converge: 60
Chi2: 478588.00748736283
Submap 35-40: FGraph initial error: 494141.67341118737
Iteratios to converge: 36
Chi2: 491797.23123139696
Submap 40-45: FGraph initial error: 860576.0451718922
Iteratios to converge: 39
Chi2: 825370.3871111659
Submap 45-50: FGraph initial error: 46564.08831214292
Itera

In [87]:
break_on_minimaps(clouds, poses, minimap_size=5, adaptive_voxelisation_iteration=1)

Submap 0-5: FGraph initial error: 94.13750202794645
Iteratios to converge: 7
Chi2: 93.41418575470136
Submap 5-10: FGraph initial error: 2787.98409933585
Iteratios to converge: 0
Chi2: 2128.259229705898
Submap 10-15: FGraph initial error: 5397.56515618583
Iteratios to converge: 43
Chi2: 885.6286834233518
Submap 15-20: FGraph initial error: 13893.85622700734
Iteratios to converge: 36
Chi2: 4426.863293249579
Submap 20-25: FGraph initial error: 13491.358041704916
Iteratios to converge: 77
Chi2: 5862.765683015978
Submap 25-30: FGraph initial error: 51871.79433437306
Iteratios to converge: 59
Chi2: 46470.37123700766
Submap 30-35: FGraph initial error: 16532.58281818496
Iteratios to converge: 75
Chi2: 4946.075430239678
Submap 35-40: FGraph initial error: 21933.647501313015
Iteratios to converge: 45
Chi2: 12354.316801628886
Submap 40-45: FGraph initial error: 23976.792440574445
Iteratios to converge: 66
Chi2: 13114.37079069578
Submap 45-50: FGraph initial error: 6396.946136695419
Iteratios to 